# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hw07\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hw07\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hw07\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import pickle

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

In [3]:
# load data from database
engine = create_engine('sqlite:///' + 'DisasterResponse.db')
df = pd.read_sql_table('dis_res_table', engine)
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    # url regular expressions
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Find all urls in the provided text
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with url placeholder string
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Extract the word tokens from the provided text
    tokens = word_tokenize(text)
    
    # Init the Wordnet Lemmatizer (converting a word to its base form)
    lemmatizer = WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
            
        ])),

        ('clf', MultiOutputClassifier(LogisticRegression(max_iter=200)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
pipeline_fitted = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
for ind, cat in enumerate(y_test):
    print('Class - {}'.format(cat))
    print(classification_report(y_test.values[ind], y_pred_test[ind], zero_division = 1))
    
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values, zero_division = 1))

Class - related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        30
           1       1.00      0.60      0.75         5

    accuracy                           0.94        35
   macro avg       0.97      0.80      0.86        35
weighted avg       0.95      0.94      0.94        35

Class - request
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        31
           1       1.00      0.50      0.67         4

    accuracy                           0.94        35
   macro avg       0.97      0.75      0.82        35
weighted avg       0.95      0.94      0.93        35

Class - offer
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        34
           1       1.00      0.00      0.00         1

    accuracy                           0.97        35
   macro avg       0.99      0.50      0.49        35
weighted avg       0.97     

In [12]:
for ind, cat in enumerate(y_test):
    print('Class - {}'.format(cat))
    print(classification_report(y_test.values[ind], y_pred_test[ind], zero_division = 1))

print("\nBest Parameters:", model.best_params_)

Class - related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        30
           1       1.00      0.60      0.75         5

    accuracy                           0.94        35
   macro avg       0.97      0.80      0.86        35
weighted avg       0.95      0.94      0.94        35

Class - request
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        31
           1       1.00      0.50      0.67         4

    accuracy                           0.94        35
   macro avg       0.97      0.75      0.82        35
weighted avg       0.95      0.94      0.93        35

Class - offer
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        34
           1       1.00      0.00      0.00         1

    accuracy                           0.97        35
   macro avg       0.99      0.50      0.49        35
weighted avg       0.97     

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
LogisticRegression().get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [16]:
parameters = {#'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
              #'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
              #'features__text_pipeline__vect__max_features': (None, 5000, 10000),
              #'features__text_pipeline__tfidf__use_idf': (True, False),
              'clf__estimator__max_iter': [200, 300],
              'clf__estimator__C': [0.5, 1.0]}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x000002255F547EE0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=LogisticRegression(max_iter=200)))]),
             param_grid={'clf__estimator__C': [0.5, 1.0],
                         'clf__estimator__max_iter': [200, 300]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
# Get the prediction values from the grid search cross validator
y_pred_test = cv.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values, zero_division = 1))

# by each class in a category
for ind, cat in enumerate(y_test):
    print('Class - {}'.format(cat))
    print(classification_report(y_test.values[ind], y_pred_test[ind], zero_division = 1))

print("\nBest Parameters:", cv.best_params_)

                        precision    recall  f1-score   support

               related       0.84      0.96      0.89      5007
               request       0.82      0.55      0.66      1142
                 offer       1.00      0.00      0.00        33
           aid_related       0.77      0.69      0.72      2705
          medical_help       0.67      0.17      0.27       507
      medical_products       0.80      0.16      0.27       335
     search_and_rescue       0.89      0.05      0.09       176
              security       1.00      0.00      0.00       109
              military       0.68      0.11      0.19       203
                 water       0.83      0.51      0.63       440
                  food       0.83      0.60      0.70       736
               shelter       0.84      0.45      0.59       582
              clothing       0.67      0.12      0.20       116
                 money       0.83      0.06      0.12       155
        missing_people       1.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

class message_lengths_words(BaseEstimator, TransformerMixin):
    
    """
    Estimator that takes a column of strings, and converts them into a column
    of integers, representing the word count of each string
    """
        
    def message_length_words(self, text):
      # tokenize by words, how many words in message
        word_list_tok = word_tokenize(text)

        return len(word_list_tok)

      
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply length_word function to all values in X
        
        X_tagged_words = pd.Series(X).apply(self.message_length_words)

        return pd.DataFrame(X_tagged_words)
     
    
class message_length_char(BaseEstimator, TransformerMixin):
    
    """
    Estimator that takes a column of strings, and converts them into a column
    of integers, representing the character length of each string
    """
    
    #get how many characters in string
    def message_length_chars(self, text):
          
        tran = len(text)
        return tran
      
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply length_char function to all values in X
        X_tagged_char = pd.Series(X).apply(self.message_length_chars)  

        return pd.DataFrame(X_tagged_char) 

In [21]:
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('word_counts', message_lengths_words()),
            ('char_lengths', message_length_char()),
            ('starting_verb_transformer', StartingVerbExtractor())
            
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

parameters = {#'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
              #'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
              #'features__text_pipeline__vect__max_features': (None, 5000, 10000),
              #'features__text_pipeline__tfidf__use_idf': (True, False),
              'clf__estimator__learning_rate': [0.01, 0.02, 0.05],
              'clf__estimator__n_estimators': [10, 20, 40]}

cv2 = GridSearchCV(pipeline2, param_grid=parameters)
cv2.fit(X_train, y_train)

# Get the prediction values from the grid search cross validator
y_pred_test = cv2.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values,  zero_division = 1))

# by each class in a category
for ind, cat in enumerate(y_test):
    print('Class - {}'.format(cat))
    print(classification_report(y_test.values[ind], y_pred_test[ind], zero_division = 1))

print("\nBest Parameters:", cv2.best_params_)

                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      5007
               request       0.87      0.23      0.37      1142
                 offer       0.00      0.00      0.00        33
           aid_related       0.80      0.26      0.39      2705
          medical_help       0.80      0.01      0.02       507
      medical_products       0.80      0.01      0.02       335
     search_and_rescue       1.00      0.01      0.01       176
              security       0.00      0.00      0.00       109
              military       0.00      0.00      0.00       203
                 water       0.85      0.47      0.61       440
                  food       0.79      0.70      0.74       736
               shelter       0.83      0.30      0.44       582
              clothing       0.00      0.00      0.00       116
                 money       1.00      0.01      0.01       155
        missing_people       0.00      

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [22]:
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.